<a href="https://colab.research.google.com/github/dami-laare/crossover-business-solutions-ai/blob/main/Solve_Business_Problems_with_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

### Configure OpenAI API Key.

In [399]:
# ===== INSTALL NECESSARY PACKAGES ====
%pip install openai httpx==0.27.2 langchain langchain pydantic langchain-core langchain-text-splitters faiss-cpu langchain-openai langchain-community  gspread gspread_dataframe

# Task 1. Classify emails

In [400]:
# Fashion Store Email Processing System
# Advanced AI-powered solution using LLMs, RAG, and Vector Stores

# imports and initial setup
import pandas as pd
import numpy as np
import re
from typing import List, Dict, Tuple, Optional, Any
import openai
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import logging
import os
import json
from dataclasses import dataclass
from google.colab import userdata
from collections import defaultdict
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load openai key from secrets to avoid hardcoded values
openai_api_key = userdata.get('OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = openai_api_key or "your-key-if-not-using-secrets"

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    max_tokens=1000,
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
)

In [401]:
# ===== CREATE GOOGLE CREDENTIALS ====
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# ===== CREATE OUTPUT DOCUMENT ====
output_document = None
try:
  output_document = gc.open('Solving Business Problems with AI - Solution Output')
except:
  output_document =  gc.create('Solving Business Problems with AI - Solution Output')

def sheet_exists(spreadsheet, sheet_name):
    """
    Check if a sheet with the given name exists in the spreadsheet

    Args:
        spreadsheet: gspread Spreadsheet object
        sheet_name: Name of the sheet to check

    Returns:
        bool: True if sheet exists, False otherwise
    """
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        return sheet
    except gspread.WorksheetNotFound:
        return False

def write_to_sheet(name: str, columns: List[str], df:pd.DataFrame, range: str):
  """
    Utility function which writes data to a google sheet

    Args:
        name: Title of the sheet
        columns: List of column headings
        df: DataFrame object which is used to populate the sheet
    """
  sheet_exists_result = sheet_exists(output_document, name)

  sheet=None
  if sheet_exists_result:
    sheet = output_document.get_worksheet_by_id(sheet_exists_result.id)
    sheet.clear()
  else:
    sheet = output_document.add_worksheet(title=name, rows=len(emails), cols=2)

  sheet.update([columns], range)


  # Write DataFrame to sheet (starting from row 2 to preserve headers)
  set_with_dataframe(sheet, df, row=2, include_index=False, include_column_header=False)

In [402]:
# ===== CREATE DATACLASSES ====
@dataclass
class Product:
    """Product data structure"""
    product_id: str
    name: str
    category: str
    description: str
    stock: int
    seasons: str
    price: float

@dataclass
class Email:
    """Email data structure"""
    email_id: str
    subject: str
    body: str
    category: Optional[str] = None

@dataclass
class OrderItem:
    """Order item data structure"""
    email_id: str
    product_id: str
    quantity: int
    status: str

In [403]:
# ===== CREATE EMAIL CLASSIFIER CLASS ====
class EmailClassifier:
    """Advanced email classifier using LLM"""

    def __init__(self, llm):
        self.llm = llm
        self.classification_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert email classifier for a fashion retail store.

            Classify emails into exactly two categories:
            1. "product inquiry" - Customer seeking information, recommendations, or clarification
            2. "order request" - Customer ready to purchase with specific product details

            CLASSIFICATION RULES:

            "order request" ONLY when ALL of these are true:
            - Customer explicitly states intent to order/buy/purchase
            - Customer provides specific product details (name, ID, or clear description)
            - Customer is ready for immediate transaction

            "product inquiry" for everything else, including:
            - Vague purchase requests without specific products ("that popular item", "bestseller")
            - Questions about products, features, availability, pricing
            - Requests for recommendations or comparisons
            - Future purchase intentions ("thinking about buying", "might order")
            - Unclear or ambiguous requests needing clarification

            EXAMPLES:

            "order request":
            - "I want to order the Versatile Scarf VSC6789"
            - "Please send me 2 Cozy Shawls"
            - "I'd like to buy your versatile scarf that can be worn as a headwrap"

            "product inquiry":
            - "What's your most popular item?"
            - "Tell me about your scarves"
            - "I want that bestselling product" (too vague)
            - "Do you have anything for winter?"
            - "I'm interested in your products"
            - "That item everyone's buying" (needs clarification)

            CRITICAL: When in doubt, classify as "product inquiry". It's better to ask for clarification than to process a vague order incorrectly.

            Respond with ONLY: "product inquiry" or "order request" """),
            ("human", "Subject: {subject}\nBody: {body}")
        ])
        self.chain = self.classification_prompt | llm

    def classify_email(self, email: Email) -> str:
        """Classify a single email"""
        try:
            result = self.chain.invoke({"subject": email.subject, "body": email.body}).content.strip()
            classification = result.strip().lower()

            # Ensure valid classification
            if "order request" in classification:
                return "order request"
            elif "product inquiry" in classification:
                return "product inquiry"
            else:
                # Fallback classification logic
                order_keywords = ['order', 'buy', 'purchase', 'send me', 'place order', 'want to order']
                body_lower = email.body.lower()
                subject_lower = email.subject.lower()

                if any(keyword in body_lower or keyword in subject_lower for keyword in order_keywords):
                    return "order request"
                else:
                    return "product inquiry"

        except Exception as e:
            logger.error(f"Error classifying email {email.email_id}: {e}")
            return "product inquiry"  # Default fallback

In [404]:
# ===== CREATE PRODUCT RAG SYSTEM CLASS ====
class ProductRAGSystem:
    """RAG system for product information retrieval"""

    def __init__(self, products: List[Product]):
        self.products = {p.product_id: p for p in products}
        self.embeddings = OpenAIEmbeddings(
            base_url="https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1"
        )
        self.setup_vector_store(products)

    def setup_vector_store(self, products: List[Product]):
        """Create vector store from product data"""
        documents = []

        for product in products:
            # Create rich document content for better retrieval
            content = f"""
            Product ID: {product.product_id}
            Name: {product.name}
            Category: {product.category}
            Description: {product.description}
            Seasons: {product.seasons}
            Price: ${product.price}
            Stock: {product.stock} units available
            """

            metadata = {
                'product_id': product.product_id,
                'name': product.name,
                'category': product.category,
                'seasons': product.seasons,
                'price': product.price,
                'stock': product.stock
            }

            documents.append(Document(page_content=content, metadata=metadata))

        # Create vector store
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100
        )
        split_docs = text_splitter.split_documents(documents)
        self.vector_store = FAISS.from_documents(split_docs, self.embeddings)
        print(f"Product vector store created with {len(documents)} products")

    def search_products(self, query: str, k: int = 5) -> List[Dict]:
        """Search for relevant products using vector similarity"""
        try:
            results = self.vector_store.similarity_search_with_score(query, k=k)

            relevant_products = []
            for doc, score in results:
                product_id = doc.metadata.get('product_id')
                if product_id in self.products:
                    product = self.products[product_id]
                    relevant_products.append({
                        'product': product,
                        'relevance_score': score,
                        'content': doc.page_content
                    })

            return relevant_products
        except Exception as e:
            logger.error(f"Error searching products: {e}")
            return []

In [405]:
# ===== CREATE UTILITY FUNCTION TO READ FROM SOURCE CSV ====
def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'


def load_data_from_sheets(sheet_id: str) -> Tuple[List[Product], List[Email]]:
    """Load data from Google Sheets"""
    try:
        # Load products
        products_df = read_data_frame(sheet_id, 'products')
        products = []
        for _, row in products_df.iterrows():
            products.append(Product(
                product_id=str(row['product_id']),
                name=str(row['name']),
                category=str(row['category']),
                description=str(row['description']),
                stock=int(row['stock']),
                seasons=str(row['seasons']),
                price=float(row['price'])
            ))

        emails_df = read_data_frame(sheet_id, 'emails')
        emails = []
        for _, row in emails_df.iterrows():
            emails.append(Email(
                email_id=str(row['email_id']),
                subject=str(row.get('subject', '')),
                body=str(row.get('message', ''))
            ))

        return products, emails

    except Exception as e:
        print(f"Error loading from Google Sheets: {e}")
        print("Falling back to empty array")
        return [], []



In [406]:
# ===== LOAD DATA FROM SOURCE CSV ====
products, emails = load_data_from_sheets(document_id)

In [407]:
# ===== CLASSIFY EMAILS ====
classifier = EmailClassifier(llm)

email_classifications = [classifier.classify_email(email) for email in emails]

for email, classification in zip(emails, email_classifications):
    email.category = classification

# ===== GROUP EMAILS BY CATEGORY ====
groups = defaultdict(list)

for email in emails:
    key = email.category    # or getattr(email, 'category')
    groups[key].append(email)

product_inquiry_emails = groups['product inquiry']
order_request_emails = groups['order request']

In [408]:
# ===== POPULATE EMAIL CLASSIFICATION SHEET ====
email_classification_data = [[email.email_id, email.category] for email in emails]
email_classification_df = pd.DataFrame(email_classification_data, columns=['email ID', 'category'])
write_to_sheet("email-classification", ['email ID', 'category'], email_classification_df, 'A1:B1')

# Task 2. Process order requests

In [409]:
# ===== CREATE INSTANCE OF PRODUCT RAG SYSTEM ====
product_rag_system = ProductRAGSystem(products=products)


Product vector store created with 99 products


In [410]:
# ===== CREATE ORDER PROCESSOR CLASS ====
class OrderProcessor:
    """Advanced order processing system"""

    def __init__(self, products: List[Product], llm):
        self.products = {p.product_id: p for p in products}
        self.stock_levels = {p.product_id: p.stock for p in products}
        self.llm = llm

        # Order extraction prompt
        self.extraction_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an expert order processor for a fashion retail store.

            Extract order information from customer emails by carefully matching against the product catalog.

            PRODUCT CATALOG FORMAT:
            Each product follows this pattern: product_id:ID product_name:NAME price:(PRICE) description:DESCRIPTION

            Available products:
            {product_catalog}

            CRITICAL: Only extract items that are CURRENT ORDER REQUESTS, not past purchases or future intentions.

            TEMPORAL ANALYSIS (Critical):
            - EXTRACT: "I want to order", "Please send me", "I'd like to buy" (present tense)
            - IGNORE: "I bought before", "I purchased", "I'm impressed with X I bought" (past tense)
            - IGNORE: "next time", "maybe later", "probably next time" (future tense)

            MATCHING INSTRUCTIONS:
            1. Read the customer email carefully
            2. Look through each product in the catalog to find matches
            3. Match based on:
              - Product IDs which can be provided as VSC6789 or in square brackets with random spacing [CBT 89 01]
              - Product IDs will always be three letters followed by 4 numbers
              - Product names (exact or partial, handle plural/singular)
              - Features mentioned in descriptions
              - Use cases described
              - Synonyms and related terms

            For the given email about "Versatile Scarves" that can be "worn as a scarf, shawl, or headwrap":
            - This should match product_id:VSC6789 product_name:Versatile Scarf because its description says "can be worn as a scarf, shawl, or even a headwrap"
            - "Versatile Scarves" (plural) matches "Versatile Scarf" (singular)

            QUANTITY RULES:
            - Explicit numbers: use that number
            - "all remaining": use -1
            - "a few", "several": use 3
            - "couple", "pair": use 2
            - No quantity specified: use 1
            - Validate: if quantity > 100, you probably used product ID by mistake

            OUTPUT FORMAT (JSON only, no markdown):
            [{{"product_id": "exact_id_from_catalog", "email_id": "exact_id_from_email", "quantity": "exact_quantity_from_the_email", "product_name": "exact_name_from_catalog", "match_confidence": "high|medium|low", "match_reason": "explanation"}}]

            EXAMPLE FOR THIS SPECIFIC CASE:
            Email: "Hello, I'd want to order one of your Versatile Scarves, the one that can be worn as a scarf, shawl, or headwrap. Thanks!"

            Analysis:
            - Customer wants "Versatile Scarves" (plural form)
            - Mentions "scarf, shawl, or headwrap" usage
            - Looking at catalog: VSC6789 "Versatile Scarf" description mentions "can be worn as a scarf, shawl, or even a headwrap"
            - Perfect match!

            Expected Response:
            [{{"product_id": "VSC6789", "quantity": 1, "product_name": "Versatile Scarf", "match_confidence": "high", "match_reason": "matches versatile scarf with headwrap/shawl capability"}}]

            EXAMPLE OF MULTIPLE PRODUCTS IN ONE:
            EMAIL: Hi, this is Liz. Please send me 5 CLF2109 Cable Knit Beanies and 2 pairs of FZZ1098 Fuzzy Slippers. I'm prepping some holiday gift baskets.

            Analysis:
            - Customer wants 2 products "CLF2109" in 5 pieces and "FZZ1098" in two places
            - The customer said 2 pairs for FZZ1098 but is talking about shoes and shoes are reffered to in pairs

            Expected Response:
            [{{"product_id": "FZZ1098", "quantity": 2, "product_name": "Fuzzy Slippers", "match_confidence": "high", "match_reason": "user provided the product ID"}}, {{"product_id": "CLF2109", "quantity": 5, "product_name": "Cable Knit Beanies", "match_confidence": "high", "match_reason": "user provided the product ID"}}]

            VALIDATION CHECKLIST:
            Before extracting each item, verify:
            1. Is this a current order request (not past purchase or future intent)?
            2. Is the quantity from the email text (not product ID)?
            3. If the quantity for the order is greater than 100 and can be found in the product ID. You got it wrong
            4. Does the customer actually want this item RIGHT NOW?),

            CRITICAL: You must find this match. The customer is clearly asking for the Versatile Scarf (VSC6789) based on the features they described."""),
            ("human", "Email Subject: {subject}\nEmail Body: {body}\nEmail ID: {email_id}")
        ])

        self.response_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a professional customer service representative for a fashion retail store.. Your name is Avery

            Generate appropriate email responses based on order processing results.

            Guidelines:
            - Professional, friendly, and personalized tone
            - Include specific product details when relevant
            - For successful orders: confirm details and thank customer
            - For out-of-stock items: apologize and suggest alternatives
            - For partial fulfillment: explain what's available
            - Keep responses concise but informative
            - Address the customer by name if mentioned
            - Do not use name placeholders. Use actul dummy data.
            """),
            ("human", """
            Customer Email Subject: {subject}
            Customer Email Body: {body}
            Order Processing Results: {order_results}
            Available Products for Alternatives: {alternatives}

            Generate a professional response email.
            """)
        ])

    def extract_product_orders(self, email: Email) -> List[Dict]:
        """Extract product orders from email using LLM"""
        try:
            product_options = product_rag_system.search_products(f"{email.subject} {email.body}", k=1)

            catalog_summary = "\n".join([
                f"product_id:{alt['product'].product_id}\nproduct_name:{alt['product'].name}\nprice:(${alt['product'].price})\ndescription:{alt['product'].description}\n\n"
                for alt in product_options[:3]
            ])

            chain = self.extraction_prompt | self.llm

            result = chain.invoke({"product_catalog": catalog_summary, "subject": email.subject, "body": email.body, "email_id": email.email_id}).content.strip()

            # Parse JSON result
            try:
                orders = json.loads(result)
                if not isinstance(orders, list):
                    orders = [orders]
                return orders
            except json.JSONDecodeError:
                # Fallback: manual extraction
                return self._manual_extraction(email)

        except Exception as e:
            logger.error(f"Error extracting orders from {email.email_id}: {e}")
            return self._manual_extraction(email)

    def _manual_extraction(self, email: Email) -> List[Dict]:
        """Fallback manual extraction using regex and keyword matching"""
        orders = []
        text = f"{email.subject} {email.body}".upper()

        # Common product ID patterns
        product_patterns = [
            r'([A-Z]{3}\s*\d{4})',  # ABC1234 or ABC 1234
            r'([A-Z]{3}\d{4})'       # ABC1234
        ]

        found_products = set()
        for pattern in product_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                clean_id = match.replace(' ', '')
                if clean_id in self.products:
                    found_products.add(clean_id)

        # Extract quantities
        quantity_patterns = [
            r'(\d+)\s*(?:pairs?\s+of|pieces?\s+of|\s+)',
            r'(?:order|buy|purchase)\s+(\d+)',
            r'(\d+)\s*(?:of\s+)?(?:the\s+)?[A-Z]{3}',
        ]

        quantities = []
        for pattern in quantity_patterns:
            matches = re.findall(pattern, text)
            quantities.extend([int(m) for m in matches])

        # Special cases
        if 'ALL REMAINING' in text or 'ALL THE REMAINING' in text:
            quantities = [-1]  # Special marker for all remaining

        # Combine products and quantities
        for i, product_id in enumerate(found_products):
            quantity = quantities[i] if i < len(quantities) else 1
            orders.append({
                'product_id': product_id,
                'quantity': quantity,
                'product_name': self.products[product_id].name
            })

        return orders

    def process_orders(self, email: Email, extracted_orders: List[Dict]) -> Tuple[List[OrderItem], str]:
        """Process extracted orders and update stock"""
        order_items = []
        results_summary = []

        for order in extracted_orders:
            product_id = order['product_id']
            requested_qty = order['quantity']

            if product_id not in self.products:
                continue

            current_stock = self.stock_levels[product_id]

            # Handle "all remaining" requests
            if requested_qty == -1:
                requested_qty = current_stock

            if current_stock >= requested_qty:
                # Order can be fulfilled
                self.stock_levels[product_id] -= requested_qty
                status = "created"
                results_summary.append(f"✓ {requested_qty}x {self.products[product_id].name} - Order Created")
            else:
                # Insufficient stock
                status = "out of stock"
                results_summary.append(f"✗ {requested_qty}x {self.products[product_id].name} - Out of Stock (Available: {current_stock})")

            order_items.append(OrderItem(
                email_id=email.email_id,
                product_id=product_id,
                quantity=requested_qty,
                status=status
            ))

        return order_items, "\n".join(results_summary)

    def generate_order_response(self, email: Email, order_results: str, rag_system: ProductRAGSystem) -> str:
        """Generate customer response email"""
        try:
            # Get alternative products for out-of-stock items
            alternatives = rag_system.search_products(f"{email.subject} {email.body}", k=3)
            alternatives_text = "\n".join([
                f"- {alt['product'].name} (${alt['product'].price}) - {alt['product'].description[:100]}..."
                for alt in alternatives[:3]
            ])

            chain = self.response_prompt | self.llm

            response = chain.invoke({"subject": email.subject, "body": email.body, "order_results": order_results, "alternatives": alternatives_text}).content.strip()

            return response

        except Exception as e:
            logger.error(f"Error generating response for {email.email_id}: {e}")
            return "Thank you for your order. We are processing your request and will get back to you shortly."


In [411]:
# ===== PROCESS ORDERS ====
order_processor = OrderProcessor(products, llm)

orders = [order_processor.extract_product_orders(email) for email in order_request_emails]

email_orders = [{"email": email, "orders": order} for email, order in zip(order_request_emails, orders)]

order_results = []
for email, order in zip(order_request_emails, orders):
    order_items, results_summary = order_processor.process_orders(email, order)
    order_results.append({
        "email": email,
        "items": order_items,
        "results_summary":results_summary,
        "response": order_processor.generate_order_response(email, str(order_items), product_rag_system)
    })

print(order_results)

# ===== POPULATE ORDER STATUS SHEET ====
order_statuses_data = [[order_result.get("email").email_id, item.product_id, item.quantity, item.status] for order_result in order_results for item in order_result.get("items")]
order_statuses_df = pd.DataFrame(order_statuses_data, columns=['email ID','product ID','quantity', 'status'])
write_to_sheet("order-status", ['email ID','product ID','quantity', 'status'], order_statuses_df, 'A1:D1')

# ===== POPULATE ORDER RESPONSES SHEET ====
order_responses_data = [[order_result.get("email").email_id, order_result.get("response")] for order_result in order_results]
order_responses_df = pd.DataFrame(order_responses_data, columns=['email ID', 'response'])
write_to_sheet("order-response", ['email ID', 'response'], order_responses_df, 'A1:B1')

[{'email': Email(email_id='E001', subject='Leather Wallets', body="Hi there, I want to order all the remaining LTH0976 Leather Bifold Wallets you have in stock. I'm opening up a small boutique shop and these would be perfect for my inventory. Thank you!", category='order request'), 'items': [OrderItem(email_id='E001', product_id='LTH0976', quantity=4, status='created')], 'results_summary': '✓ 4x Leather Bifold Wallet - Order Created', 'response': "Subject: Confirmation of Your Leather Wallet Order\n\nHi there,\n\nThank you for reaching out and for your interest in our LTH0976 Leather Bifold Wallets for your boutique shop. I'm pleased to confirm that we have successfully processed your order for the remaining 4 wallets in stock.\n\nThese wallets are a fantastic choice and I'm confident they will be a great addition to your inventory. If you need any further assistance or have any questions, please feel free to reach out.\n\nThank you for choosing our products for your boutique. We look 

# Task 3. Handle product inquiry

In [412]:
# ===== CREATE INQUIRY HANDLER CLASS ====
class InquiryHandler:
    """Handle product inquiries using RAG"""

    def __init__(self, rag_system: ProductRAGSystem, llm):
        self.rag_system = rag_system
        self.llm = llm

        self.inquiry_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a knowledgeable fashion retail assistant. Your name is Avery

            Help customers with their product inquiries using the provided product information.

            Guidelines:
            - Be helpful, friendly, and professional
            - Provide specific product details when available
            - Make personalized recommendations based on customer needs
            - Include pricing and availability information
            - Suggest alternatives when appropriate
            - Keep responses concise but informative
            - Address the customer by name if mentioned
            - Do not use name placeholders. Use actul dummy data.
            """),
            ("human", """
            Customer Email Subject: {subject}
            Customer Email Body: {body}

            Relevant Products:
            {relevant_products}

            Generate a helpful response to the customer's inquiry.
            """)
        ])

    def handle_inquiry(self, email: Email) -> str:
        """Handle product inquiry and generate response"""
        try:
            # Search for relevant products
            relevant_products = self.rag_system.search_products(
                f"{email.subject} {email.body}", k=5
            )

            # Format product information
            products_info = []
            for item in relevant_products:
                product = item['product']
                products_info.append(f"""
                - {product.name} ({product.product_id})
                  Category: {product.category}
                  Price: ${product.price}
                  Description: {product.description}
                  Seasons: {product.seasons}
                  Stock: {product.stock} available
                """)

            products_text = "\n".join(products_info)

            chain = self.inquiry_prompt | self.llm

            response = chain.invoke({"subject": email.subject, "body": email.body, "relevant_products": products_text}).content.strip()

            return response

        except Exception as e:
            logger.error(f"Error handling inquiry for {email.email_id}: {e}")
            return "Thank you for your inquiry. We'll get back to you with detailed information shortly."


In [413]:
# ===== PROCESS INQUIRIES ====
inquiry_handler = InquiryHandler(product_rag_system, llm)

inquiry_responses = [{"email": email,"response": inquiry_handler.handle_inquiry(email)} for email in product_inquiry_emails]

# ===== POPULATE INQUIRY RESPONSES SHEET ====
inquiry_responses_data = [[inquiry_response.get("email").email_id, inquiry_response.get("response")] for inquiry_response in inquiry_responses]
inquiry_responses_df = pd.DataFrame(inquiry_responses_data, columns=['email ID', 'response'])
write_to_sheet("inquiry-response", ['email ID', 'response'], inquiry_responses_df, 'A1:B1')